In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, timezone
from urllib.parse import urlparse
from dateutil import parser
import unicodedata
import spacy
import numpy as np
import os

# === CONFIGURACIÓN GENERAL ===
terminos_busqueda = ["Parque Industrial","Nave Industrial","Parques Industriales","Naves industriales","hub industrial","arancel","aranceles","inversión inmobiliaria",
                    "hub logístico","Fibra","Real Estate Industrial","mercado industrial","mercados industriales","Sector Industrial",
                    "construcción industrial","fusión","fusiones", "automotriz","construcción","transporte","logística","sector logístico","cadena de suministro","transporte de carga","autopartes",
                    "tasas de interés","La Fed", "reserva federal","banco central"]
modo = "rango"  # Opciones: "exacto" o "rango"
debug = True     # True para imprimir detalles por noticia
dias_rango = 1  # Solo si modo = "rango"
fecha_objetivo = datetime(2025, 7, 3).date()  # Solo si modo = "exacto"

medios = ["eleconomista.com.mx","imagenradio.com.mx","elfinanciero.com.mx","forbes.com.mx","merca20.com",
    "eluniversal.com.mx","heraldodemexico.com.mx/nacional","nexos.com.mx","thelogisticsworld.com","t21.com.mx",
    "articulo19.org","animalpolitico.com","sinembargo.mx","codigomagenta.com.mx","latinus.us","expansion.mx",
    "nmas.com.mx/nacional","radioformula.com.mx","wradio.com.mx","unotv.com","tvazteca.com/aztecanoticias","infobae.com/mexico",
    "la-lista.com/mexico","oem.com.mx/la-prensa","oem.com.mx/elsoldemexico","proceso.com.mx","vertigopolitico.com","sdpnoticias.com",
    "lasillarota.com/nacion","excelsior.com.mx","letraslibres.com","elpais.com/mexico/","mvsnoticias.com","latimes.com","jornada.com.mx",
    "cnnespanol.cnn.com/","nytimes.com","mexico.quadratin.com.mx","milenio.com","informador.mx","washingtonpost.com",
    "realestatemarket.com.mx/mercado-inmobiliario-home/industrial","alfamexico.com/category/noticias-inmobiliarias/",
    "inmobiliare.com/","jll.com.mx/es/newsroom","eleconomista.com.mx/tags/sector-inmobiliario","milenio.com/temas/sector-inmobiliario",
    "somosindustria.com/", "mexicoindustry.com/","thelogisticsworld.com/","bloomberglinea.com/latinoamerica/mexico/"
]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
}

nlp = spacy.load("es_core_news_sm")

# Funciones auxiliares
def normalizar(texto):
    if not isinstance(texto, str):
        return ""
    texto = unicodedata.normalize("NFKD", texto)
    texto = texto.encode("ascii", "ignore").decode("utf-8")
    return texto.lower()

def extraer_adjetivos(texto):
    doc = nlp(str(texto))
    return [token.text for token in doc if token.pos_ == "ADJ"]

categorias = {"Interior": ["Rosa Icela Rodríguez", "SEGOB", "segob", "Secretaría de Gobernación", "corridas de toros","taurino", "tauromaquia", "ISSSTE",
                 "CNTE","SNTE","bloqueos","marchas","bloqueo","marcha", "INAI","Rocha Moya", "Mexicable", "Bienestar", "Sindicato",
                "jefe de gabinete", "Martí Batres", "gabinete", "fiesta brava", "San Lázaro","Cámara de Diputados","Sismo en México" 
                 "congreso", "partido", "senador", "gobernador", "senadora", "gobernadora", "Senado", "diputado","diputados","senadores", 
                 "PROFECO", "Infonavit","Sembrando Vida","desafuero","derechos laborales","reforma laboral","líder sindical","Noroña","Alito",
                 "Ricardo Anaya","Morena","PRI","PAN","Movimiento Ciudadano", "Comisión de Igualdad de Género","jubilación","Marko Cortés",
                 "Jóvenes construyendo","ley","leyes","reforma","reformas", "Kuri", "Menchaca","Lemus","Samuel García","UdeG","UNAM",
                "Evelyn Salgado","parlamento abierto","Mara Lezama","jucopo","Murat","morenistas","yunes","Monreal","Adán Augusto",
                "Armenta","PAN","Ivonne Ortega", "cuauhtémoc", "cuau", "pedro haces","ramírez cuellar","marín","rectora","ealy","desalojo",
                "colosio","anaya","Lajous","pensión","Ricardo Gallardo", "Programa alimentario","agroindustria", "Ramírez Bedolla","paro nacional",
                          "ventanilla digital", "conservadores", "asuntos internos", "Brugada","cdmx", "cablebús","Línea 1", "simulacro"],
    
    "Corrupción": ["corrupción", "corrupto", "desfalco", "soborno", "coludido","Odebrecht","sobreprecios","sobreprecio","ASF","Auditoría Superior de la Federación",
                  "irregularidades","sobrecosto","contratista","opacidad","compranet","Raquel Buenrostro","Andrea Chávez"],
    
    "Judicial": ["fiscalía", "cortes", "corte", "exfiscal", "audiencia", "fiscal", "extradición","juez", "jueza", "fiscalía general", "juicio",
                 "ministerio público","FGR","Lenia Batres","poder judicial","SCJN","judicial","extradición","extraditó","Aguakan","Gertz","Magistrada",
                "Tribunal Superior de Justicia","TSJ","Inés Gómez Mont", "Caso Zimapán","UIF","Yasmín Esquivel","Lenia Batres","Loreta Ortíza"],
    
    "Electoral": ["elección", "campaña", "elecciones", "candidato", "candidatos", "candidata","TEPJF","OPLES","INE","aprehenden","candidaturas"],  
    
    "Medio Ambiente":["contingencia","incendio forestal","incendios forestales","deforestación","racionamiento","escasez de agua","sin agua",
                      "ozono","contaminación","crisis climática","desastres climáticos","huracán","huracanes","corte de agua","calidad del aire",
                     "vaquita marina","Áreas Naturales Protegidas","PROFEPA","polvo","aire sucio","sequía","Plan Nacional Hídrico","acceso al agua",
                      "calor extremo","agua","SEMARNAT","racionamiento","glaciares","greenpeace","totoabas","tortugas", "Frente Frío", "Medio Ambiente",
                     "temporada de calor","radiación solar"],
    
    "Exterior": ["canciller", "TLC", "Tratado de Libre Comercio", "diplomático", "diplomática", "embajada", "embajador", "TMEC", "Venezuela", "ONU", 
                 "Nicolás Maduro","OTAN", "Frontera", "deportaciones","deportados","deportado", "Zelenski","Ucrania", "Palestina", "Hamás","Hamas", "Israel", 
                 "Lula da Silva","Putin","Papa Francisco","Millei","Vaticano", "Chile","Hungría","UE","Unión Europea","Francia","Macron", "Canadá", "Guatemala",
                 "FMI","Kennedy","Donald Trump","Trump","Marco Rubio","De la Fuente","Musk","Wall Street","Fed","EU","Gaza","Estados Unidos","EEUU","Bukele",
                "speak","Estambul","Turquía","Arabia Saudita","Ted Cruz","Sahel","relación bilateral","Carney", "Kristi Noem","Lutnick","migración",
                "celac","Honduras","Petro","Xiomara","Kamala", "Colombia","Von Der Leyen","China","chino","Corea del Sur","Mangione","Myanmar","EE.UU.",
                "Noboa","Ecuador","Noem","antiinmigrantes","SRE","Norteamérica","extranjeros", "extranjero","Emiratos Árabes"],
    
    "Economía": ["Desempleo","nearshoring","SAT","Secretaría de Hacienda","Secretario de Hacienda","Presupuesto","fiscal","arancel", "aranceles"
                "exportación","importación","exportaciones","importaciones","precios","cotización","tipo de cambio","CEO","CFE","General Motors",
                 "energética","crudo","Banxico","Banco de México", "tasas de interés","tasa de interés","peso mexicano", "armadoras", "automotriz",
                 "competitividad","calificadora","calificadoras","S&P","Moody's","Fitch","Nasdaq", "Dow Jones","Plan México","trigo","PIB","oro",
                "aduanero","aduanera","déficit","Pemex","hidrocarburos","KPMG","Deloitte","Santander","sofipos","nóminas","recesión", "empleo","remesas",
                "derrama económica","Ebrard","fintech","Banco","Bancos","DOGE","Coppel","Walmart","Oxxo","reforma energética","turistas","telmex",
                "Kueski","SENER","AIFA","T-MEC","BYD","Bolsa mexicana","deprecia","CAINTRA","ronda de inversión","inversión millonaria","Política económica","Italika",
                "AIMMGM","consumo privado","Empleo","Hongqi","inversión conjunta","Coca Cola","Topo Chico","precio de los huevos","tendrá inversión","ADM","Peso cae",
                "fovissste","Slim","tortilla","tortillas","CEO","Altagracia","recaudación fiscal","economía", "Mercedes-Benz", "Volkswagen","económico",
                "VW","Audi","BMW","stellantis","gm","general motors","dos bocas","refinería","Transístmico","tarifas","acero","aluminio", "Nissan",
                "automotrices","agrícola", "farmacéutica","interoceánico", "Amazon","crisis económica","chrysler", "salinas pliego","volvo","citi",
                "cce","shcp","infonavit","comercial","Land Rover","Jaguar","Oracle","JPMorgan Chase","Truth Social","Totalplay","fabricación de autos",
                "cierra dólar","expectativa de crecimiento","BMV","Bolsa Mexicana de Valores", "peñoles", "hacienda","unicornio", "recaudación","INEGI",
                "confianza del consumidor","cosechando soberanía","ocupación hotelera","económica","vías férreas","BM","monopolio","antimonopolio",
                "monopolios","antimonopolios","cofece","leche para el bienestar", "miel del bienestar","cepal"],
    
    "Seguridad": ["Teuchitlán","Izaguirre","cárteles","cártel","homicidios","homicidio","desaparecidos","desaparecido","CJNG","feminicidio",
                  "Tren de Aragua","Cártel de Sinaloa","armado","violencia","baleado","fentanilo","narco","comando","asesinada","asesinado","asesinados","asesinadas",
                  "asesinan","robo","robos","secuestro","secuestros","clandestino","clandestina","clandestinas","clandestinos","García Harfuch","balean",
                  "delitos","Chapitos","Mayiza","arma","Marina","violencia","Comisión Nacional de Búsqueda", "personas desaparecidas","crimen organizado",
                 "Guardia Nacional","GN","narcotraficantes","disparó","dispararon","armas","aduana","ejecutan","narcorancho","asesinar","abaten","narcotraficante",
                 "decomiso","decomisan","detienen","detención", "desaparición forzada","Los Cantaritos","desapariciones","atentado","ejecutados",
                 "tiros","reclutamiento forzado", "exterminio","rifle","fusil","drogas","explosivos","detenidos","clandestino","narcotráfico","muertos",
                 "militares","exmilitares","marinos","tiroteo","madres buscadoras","desapariciones","cuerpos","raptadas","matan","asesinato","terrorismo",
                 "Semar","Seretaría de Marina","matanza","semefo","ejecutaron","ejecutan","balaceras","capos","delincuentes","abrazos no balazos", "mencho",
                 "derecho de piso","Los Alegres del Barranco","Doña Carlota","disparando","confiscan","rearma","incautan", "Teresa González Murillo",
                 "crimen", "familia michoacana","contrabando", "Carlota N", "sin vida","operativos", "balacera","roban","El mayo","restos humanos",
                 "mutilado", "linchan","linchamiento"],
    
    "Educación": ["Acceso a educación","SEP","Mario Delgado","CCH","UNAM","IPN","universitarios","escuelas", "maestros"],
    
    "Salud": ["salud", "IMSS", "IMSS Bienestar","tos ferina","sarampión","enfermedad",  "vacuna", "médico","medicina", "clínica", "epidemia","vacunación",
              "obesidad","vacunas","medicinas", "Eduardo Clark","Kershenobich","sobrepeso", "influenza", "H5N1", "aviar","dengue", "hospitales"]
}
palabras_positivas = ["éxito", "triunfó", "logro", "logros", "avance", "progreso", "mejoría", "crecimiento","baja desempleo","baja inflación",
    "decomiso de fentanilo","Suspenden Contingencia Ambiental","suspende contingencia ambiental","Suspendida la contingencia ambiental",
    "aprobación","suma otra inversión","felicidad","Guardia Nacional asegura","GN aseguran","felices","feliz","Golpe al narco",
    "decomiso","decomisan","firmeza de Sheinbaum","detienen","aseguran dosis","aseguran cargamento","aseguran artefactos","aprehenden","detenidos",
    "elementos navales descubren","presume Sheinbaum","Sheinbaum celebra","Sheinbaum inaugura","rescatan","Inaugura Sheinbaum","invierte",
    "incentivar","Entrega Clara Brugada","tendrá inversión","anuncia inversión","bienvenidas las inversiones","impulsa el desarrollo",
    "impulsa inversión","potencia minería","revoluciona la minería","Avanza la minería","minería del futuro","Más vivo",
    "impulsa la minería","amplía meta", "impulsan innovación", "transforma","innovación","auge","repunta","construcción sostenible", 
    "construcción sustentable","innovaciones","eficiencia","avanza","detonan construcción", "Obtiene reconocimiento", "fortalece",
    "señales de recuperación","crece el sector", "impulsa","bienvenidas","bienvenida","facilidades","vanguardia","inclusión",
    "inicia la construcción", "anuncian la construcción","anuncia proyectos","iniciará su construcción", "anuncia la construcción",
    "arranca construcción", "anuncia construcción","acelerará construcción", "acelera construcción", "acuerdan construcción",
    "inicia proyecto","agradece","innovadores","inicia construcción","nuevo hospital","nuevos hospitales","resalta construcción",
    "banderazo","anuncia una inversión", "iniciará construcción", "iniciará la construcción","inicio la construcción", 
    "inician construcción","optimismo","incautan","rescatan", "decomiso","decomisan","detención", "confiscan", "detenidos",
    "inversión millonaria", "beneficiará", "celebra", "golpe al peso", "Lanzan programas","forteleció el peso","fortalece le peso",
    "peso fortalecido"]

palabras_negativas = ["fracaso", "crisis", "colapso", "escándalo", "corrupción","irresponsabilidad", "problema","deterioro", "violencia", 
    "homicidio", "irrespetuosas", "peligroso", "peligrosa", "fallida","podrido", "difícil","dura", "duro", "incongruente", 
    "retrocede", "grave","traición", "pierde", "desaprobación", "desaprueba","engaña", "engaño", "sucio", "sucia", "perdería", 
    "fallido", "mal", "malo", "mala", "débil","violación","pesadilla", "desplazamiento", "polémica", "desapariciones", "cacería",
    "polémicas", "colapsó", "ineficiente", "agujero", "fallé", "desastre","vergüenza", "aumento del desempleo", "Mencho","Chapitos",
    "Mayiza","Teuchitlán","Izaguirre","exterminio","fosa clandestina","CJNG","Cártel de Sinaloa","Activan Contingencia Ambiental",
    "chantaje","chantajea","horror","mala calidad del aire","huachicol","fuga de gas","toman casetas","secuestrado","asesinados",
    "Izaguirre","contaminación","Ayotzinapa","balacera","infierno","Cartel Jalisco Nueva Generación","obesidad","sobrepeso","bombardeo",
    "sismo","tiroteo","incendio","muere","muerte","explosión","reconoce fallas","ejecutan","polvo","bloqueo","rapta","cuerpos","atentado",
    "sequía","asesinato","contra México","sanciones a México","aranceles","arancel", "terrorismo","terrorista","matanza","ejecutaron","ejecutan",
    "Descomposición","balaceras","Peso cae","narco mexicano incursiona","estafan","retroceso","cae","afectarán","afecta",
    "recesión","riesgo","desaceleración","retrasos","daños","Myanmar", "denuncian", "afectan", "ilelgal","devastación",
    "Protesta", "retrasaría","inconformidades","rechazan","clausura","desmienten construcción","suspender","frenar construcción",
    "protestan", "protestas","repudia","repudian","recortan expectativa", "Teuchitlán","Izaguirre","cárteles","cártel","homicidios",
    "Tren de Aragua","Cártel de Sinaloa","armado","violencia","baleado","fentanilo","narco","comando","asesinada","asesinado","asesinados","asesinadas",
    "asesinan","robo","robos","secuestro","secuestros","clandestino","clandestina","clandestinas","clandestinos","balean",
    "delitos","Chapitos","Mayiza","arma","Marina","violencia","Comisión Nacional de Búsqueda", "personas desaparecidas","crimen organizado",
    "Guardia Nacional","GN","narcotraficantes","disparó","dispararon","armas","aduana","ejecutan","narcorancho","asesinar","narcotraficante",
    "desaparición forzada","Los Cantaritos","desapariciones","atentado","ejecutados","homicidio","desaparecidos",
    "tiros","reclutamiento forzado", "exterminio","rifle","fusil","drogas","explosivos","clandestino","narcotráfico","muertos",
    "militares","exmilitares","marinos","tiroteo","madres buscadoras","desapariciones","raptadas","matan","asesinato","terrorismo",
    "matanza","semefo","ejecutaron","ejecutan","balaceras","capos","delincuentes","abrazos no balazos", "mencho",
    "derecho de piso","Los Alegres del Barranco","Doña Carlota","disparando","rearma","incautan", "Teresa González Murillo",
    "crimen", "familia michoacana","contrabando", "Carlota N", "sin vida","operativos", "balacera","roban","El mayo", "desaparecido","CJNG",
    "feminicidio", "aranceles recíprocos", "tira", "escasez de agua", "restos humanos", "mutilado", "linchan","linchamiento", "Cae confianza"]

def clasificar_sentimiento(titulo, adjetivos):
    titulo_norm = normalizar(titulo)
    palabras_pos = [word for word in palabras_positivas if normalizar(word) in titulo_norm]
    palabras_neg = [word for word in palabras_negativas if normalizar(word) in titulo_norm]

    if palabras_pos and not palabras_neg:
        return "Positiva"
    elif palabras_neg and not palabras_pos:
        return "Negativa"
    else:
        return "Neutral"

def palabras_clasificadoras(titulo):
    titulo_norm = normalizar(titulo)
    return ", ".join([word for word in palabras_positivas + palabras_negativas if normalizar(word) in titulo_norm])

def categorizar_noticia(titulo):
    titulo_norm = normalizar(titulo)
    categorias_encontradas = set()
    for categoria, palabras in categorias.items():
        for palabra in palabras:
            palabra_norm = normalizar(palabra)
            if len(palabra_norm) <= 3:
                if f" {palabra_norm} " in f" {titulo_norm} ":
                    categorias_encontradas.add(categoria)
            else:
                if palabra_norm in titulo_norm:
                    categorias_encontradas.add(categoria)
    return ", ".join(categorias_encontradas) if categorias_encontradas else "Otros"

# Cargar catálogo de visitas mensuales
cat_path = "catalogo_medios_mexicanos_con reach.xlsx"
cat_df = pd.read_excel(cat_path)
cat_df["Dominio Limpio"] = cat_df["Dominio"].apply(lambda x: urlparse("https://" + str(x)).netloc.replace("www.", "").split("/")[0])
cat_df = cat_df.rename(columns={"Visitas mensuales con base en marzo 2025": "Visitas_Mensuales"})

noticias = []
ahora = datetime.now(timezone.utc)

for termino in terminos_busqueda:
    print(f"\n🔍 Buscando: {termino}")

    for medio in medios:
        dominio = urlparse("https://" + medio).netloc
        url = f"https://news.google.com/rss/search?q={termino}+site:{dominio}&hl=es-419&gl=MX&ceid=MX:es"

        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"⚠️ Error al obtener noticias de {dominio}")
            continue

        soup = BeautifulSoup(response.content, "xml")
        items = soup.find_all("item")
        if debug:
            print(f"⚙️  {dominio} → {len(items)} noticias")

        for item in items:
            title = item.title.text.strip()
            link = item.link.text.strip()
            description = item.description.text.strip() if item.description else ""
            pub_date_str = item.pubDate.text.strip()

            try:
                pub_date = parser.parse(pub_date_str)
            except Exception:
                continue

            if modo == "exacto":
                if pub_date.date() != fecha_objetivo:
                    continue
            elif modo == "rango":
                if ahora - pub_date > timedelta(days=dias_rango):
                    continue

            contenido = f"{title} {description}".lower()
            if termino.lower() in contenido:
                if debug:
                    print(f"📰 {title} | {pub_date.date()} | {dominio}")
                noticias.append([pub_date_str, title, link, dominio, termino])
# 📁 Crear carpeta si no existe
carpeta_salida = "noticias"
os.makedirs(carpeta_salida, exist_ok=True)
# Guardar resultados
if noticias:
    df = pd.DataFrame(noticias, columns=["Fecha", "Título", "Enlace", "Fuente", "Término"])
    df["Título"] = df["Título"].str.split(" - ").str[0]
    df = df.drop_duplicates(subset=["Título", "Fuente"])
    df["Fecha"] = pd.to_datetime(df["Fecha"], errors="coerce")
    df = df[df["Fecha"].notna()]
    df["Fecha"] = df["Fecha"].dt.strftime("%d/%m/%y")
    df["Adjetivos"] = df["Título"].apply(extraer_adjetivos)
    df["Sentimiento"] = df.apply(lambda row: clasificar_sentimiento(row["Título"], row["Adjetivos"]), axis=1)
    df["Palabras Clave"] = df["Título"].apply(palabras_clasificadoras)
    df["Categoría"] = df["Título"].apply(categorizar_noticia)
    df["Dominio Limpio"] = df["Fuente"].apply(lambda x: urlparse("https://" + str(x)).netloc.replace("www.", "").split("/")[0])
    df = df.merge(cat_df[["Dominio Limpio", "Visitas_Mensuales"]], on="Dominio Limpio", how="left")
    df["Reach estimado"] = df["Visitas_Mensuales"] * 0.001 * np.random.uniform(0.6, 1.2, size=len(df))

    sufijo_fecha = fecha_objetivo.strftime("%d-%m-%Y") if modo == "exacto" else ahora.strftime("%d-%m-%Y")
    nombre_archivo = os.path.join(
        carpeta_salida, f"noticias_fuentes_nacionales_{sufijo_fecha}.csv"
)
    df.to_csv(nombre_archivo, index=False, encoding="utf-8-sig")
    print(f"\n✅ Archivo guardado como '{nombre_archivo}' con {len(df)} noticias")
else:
    print("\n❌ No se encontraron noticias que cumplan los filtros.")



🔍 Buscando: Parque Industrial
⚙️  eleconomista.com.mx → 100 noticias
⚙️  imagenradio.com.mx → 35 noticias
⚙️  elfinanciero.com.mx → 100 noticias
⚙️  forbes.com.mx → 100 noticias
⚙️  merca20.com → 65 noticias
⚙️  eluniversal.com.mx → 100 noticias
⚙️  heraldodemexico.com.mx → 100 noticias
⚙️  nexos.com.mx → 92 noticias
⚙️  thelogisticsworld.com → 100 noticias
⚙️  t21.com.mx → 100 noticias
⚙️  articulo19.org → 6 noticias
⚙️  animalpolitico.com → 100 noticias
⚙️  sinembargo.mx → 100 noticias
⚙️  codigomagenta.com.mx → 14 noticias
⚙️  latinus.us → 58 noticias
⚙️  expansion.mx → 100 noticias
⚙️  nmas.com.mx → 100 noticias
⚙️  radioformula.com.mx → 100 noticias
⚙️  wradio.com.mx → 58 noticias
⚙️  unotv.com → 100 noticias
⚙️  tvazteca.com → 100 noticias
⚙️  infobae.com → 100 noticias
⚙️  la-lista.com → 56 noticias
⚙️  oem.com.mx → 100 noticias
⚙️  oem.com.mx → 100 noticias
⚙️  proceso.com.mx → 100 noticias
⚙️  vertigopolitico.com → 60 noticias
⚙️  sdpnoticias.com → 100 noticias
⚙️  lasillarot

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, timezone
from urllib.parse import urlparse
from dateutil import parser
import unicodedata
import spacy
import numpy as np
import os


# === CONFIGURACIÓN GENERAL ===
terminos_busqueda = ["Parque Industrial","Nave Industrial","Parques Industriales","Naves industriales","hub industrial","arancel","aranceles","inversión inmobiliaria",
                    "hub logístico","Fibra","Real Estate Industrial","mercado industrial","mercados industriales","Sector Industrial",
                    "construcción industrial","fusión","fusiones", "automotriz","construcción","transporte","logística","sector logístico","cadena de suministro","transporte de carga","autopartes"]
modo = "rango"  # Opciones: "exacto" o "rango"
debug = True     # True para imprimir detalles por noticia
dias_rango = 1   # Solo si modo = "rango"
fecha_objetivo = datetime(2025, 7,3).date()  # Solo si modo = "exacto"

medios = ["abcnoticias.mx/local/","abcnoticias.mx/tamaulipas/","elnorte.com/","info7.mx/","elhorizonte.mx/nuevoleon","elporvenir.mx/","elregio.com/","omareli.com/","elmanana.com/","latarde.com.mx/",
          "laprensa.mx/","valledelnorte.com.mx/sitio/","psn.si/tijuana-san-diego/","jornadabc.com.mx/","afntijuana.info/","puntonorte.info/",
          "zetatijuana.com/","diario.mx/seccion/juarez/","netnoticias.mx/juarez","puentelibre.mx/","nortedigital.mx/category/economia/",
          "adiario.mx/secciones/estado/juarez/", "juarezhoy.com.mx/estado/"
]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
}

nlp = spacy.load("es_core_news_sm")

# Funciones auxiliares
def normalizar(texto):
    if not isinstance(texto, str):
        return ""
    texto = unicodedata.normalize("NFKD", texto)
    texto = texto.encode("ascii", "ignore").decode("utf-8")
    return texto.lower()

def extraer_adjetivos(texto):
    doc = nlp(str(texto))
    return [token.text for token in doc if token.pos_ == "ADJ"]

categorias = {"Interior": ["Rosa Icela Rodríguez", "SEGOB", "segob", "Secretaría de Gobernación", "corridas de toros","taurino", "tauromaquia", "ISSSTE",
                 "CNTE","SNTE","bloqueos","marchas","bloqueo","marcha", "INAI","Rocha Moya", "Mexicable", "Bienestar", "Sindicato",
                "jefe de gabinete", "Martí Batres", "gabinete", "fiesta brava", "San Lázaro","Cámara de Diputados","Sismo en México" 
                 "congreso", "partido", "senador", "gobernador", "senadora", "gobernadora", "Senado", "diputado","diputados","senadores", 
                 "PROFECO", "Infonavit","Sembrando Vida","desafuero","derechos laborales","reforma laboral","líder sindical","Noroña","Alito",
                 "Ricardo Anaya","Morena","PRI","PAN","Movimiento Ciudadano", "Comisión de Igualdad de Género","jubilación","Marko Cortés",
                 "Jóvenes construyendo","ley","leyes","reforma","reformas", "Kuri", "Menchaca","Lemus","Samuel García","UdeG","UNAM",
                "Evelyn Salgado","parlamento abierto","Mara Lezama","jucopo","Murat","morenistas","yunes","Monreal","Adán Augusto",
                "Armenta","PAN","Ivonne Ortega", "cuauhtémoc", "cuau", "pedro haces","ramírez cuellar","marín","rectora","ealy","desalojo",
                "colosio","anaya","Lajous","pensión","Ricardo Gallardo", "Programa alimentario","agroindustria", "Ramírez Bedolla","paro nacional",
                          "ventanilla digital", "conservadores", "asuntos internos", "Brugada","cdmx", "cablebús","Línea 1", "simulacro"],
    
    "Corrupción": ["corrupción", "corrupto", "desfalco", "soborno", "coludido","Odebrecht","sobreprecios","sobreprecio","ASF","Auditoría Superior de la Federación",
                  "irregularidades","sobrecosto","contratista","opacidad","compranet","Raquel Buenrostro","Andrea Chávez"],
    
    "Judicial": ["fiscalía", "cortes", "corte", "exfiscal", "audiencia", "fiscal", "extradición","juez", "jueza", "fiscalía general", "juicio",
                 "ministerio público","FGR","Lenia Batres","poder judicial","SCJN","judicial","extradición","extraditó","Aguakan","Gertz","Magistrada",
                "Tribunal Superior de Justicia","TSJ","Inés Gómez Mont", "Caso Zimapán","UIF","Yasmín Esquivel","Lenia Batres","Loreta Ortíza"],
    
    "Electoral": ["elección", "campaña", "elecciones", "candidato", "candidatos", "candidata","TEPJF","OPLES","INE","aprehenden","candidaturas"],  
    
    "Medio Ambiente":["contingencia","incendio forestal","incendios forestales","deforestación","racionamiento","escasez de agua","sin agua",
                      "ozono","contaminación","crisis climática","desastres climáticos","huracán","huracanes","corte de agua","calidad del aire",
                     "vaquita marina","Áreas Naturales Protegidas","PROFEPA","polvo","aire sucio","sequía","Plan Nacional Hídrico","acceso al agua",
                      "calor extremo","agua","SEMARNAT","racionamiento","glaciares","greenpeace","totoabas","tortugas", "Frente Frío", "Medio Ambiente",
                     "temporada de calor","radiación solar"],
    
    "Exterior": ["canciller", "TLC", "Tratado de Libre Comercio", "diplomático", "diplomática", "embajada", "embajador", "TMEC", "Venezuela", "ONU", 
                 "Nicolás Maduro","OTAN", "Frontera", "deportaciones","deportados","deportado", "Zelenski","Ucrania", "Palestina", "Hamás","Hamas", "Israel", 
                 "Lula da Silva","Putin","Papa Francisco","Millei","Vaticano", "Chile","Hungría","UE","Unión Europea","Francia","Macron", "Canadá", "Guatemala",
                 "FMI","Kennedy","Donald Trump","Trump","Marco Rubio","De la Fuente","Musk","Wall Street","Fed","EU","Gaza","Estados Unidos","EEUU","Bukele",
                "speak","Estambul","Turquía","Arabia Saudita","Ted Cruz","Sahel","relación bilateral","Carney", "Kristi Noem","Lutnick","migración",
                "celac","Honduras","Petro","Xiomara","Kamala", "Colombia","Von Der Leyen","China","chino","Corea del Sur","Mangione","Myanmar","EE.UU.",
                "Noboa","Ecuador","Noem","antiinmigrantes","SRE","Norteamérica","extranjeros", "extranjero","Emiratos Árabes"],
    
    "Economía": ["Desempleo","nearshoring","SAT","Secretaría de Hacienda","Secretario de Hacienda","Presupuesto","fiscal","arancel", "aranceles"
                "exportación","importación","exportaciones","importaciones","precios","cotización","tipo de cambio","CEO","CFE","General Motors",
                 "energética","crudo","Banxico","Banco de México", "tasas de interés","tasa de interés","peso mexicano", "armadoras", "automotriz",
                 "competitividad","calificadora","calificadoras","S&P","Moody's","Fitch","Nasdaq", "Dow Jones","Plan México","trigo","PIB","oro",
                "aduanero","aduanera","déficit","Pemex","hidrocarburos","KPMG","Deloitte","Santander","sofipos","nóminas","recesión", "empleo","remesas",
                "derrama económica","Ebrard","fintech","Banco","Bancos","DOGE","Coppel","Walmart","Oxxo","reforma energética","turistas","telmex",
                "Kueski","SENER","AIFA","T-MEC","BYD","Bolsa mexicana","deprecia","CAINTRA","ronda de inversión","inversión millonaria","Política económica","Italika",
                "AIMMGM","consumo privado","Empleo","Hongqi","inversión conjunta","Coca Cola","Topo Chico","precio de los huevos","tendrá inversión","ADM","Peso cae",
                "fovissste","Slim","tortilla","tortillas","CEO","Altagracia","recaudación fiscal","economía", "Mercedes-Benz", "Volkswagen","económico",
                "VW","Audi","BMW","stellantis","gm","general motors","dos bocas","refinería","Transístmico","tarifas","acero","aluminio", "Nissan",
                "automotrices","agrícola", "farmacéutica","interoceánico", "Amazon","crisis económica","chrysler", "salinas pliego","volvo","citi",
                "cce","shcp","infonavit","comercial","Land Rover","Jaguar","Oracle","JPMorgan Chase","Truth Social","Totalplay","fabricación de autos",
                "cierra dólar","expectativa de crecimiento","BMV","Bolsa Mexicana de Valores", "peñoles", "hacienda","unicornio", "recaudación","INEGI",
                "confianza del consumidor","cosechando soberanía","ocupación hotelera","económica","vías férreas","BM","monopolio","antimonopolio",
                "monopolios","antimonopolios","cofece","leche para el bienestar", "miel del bienestar","cepal"],
    
    "Seguridad": ["Teuchitlán","Izaguirre","cárteles","cártel","homicidios","homicidio","desaparecidos","desaparecido","CJNG","feminicidio",
                  "Tren de Aragua","Cártel de Sinaloa","armado","violencia","baleado","fentanilo","narco","comando","asesinada","asesinado","asesinados","asesinadas",
                  "asesinan","robo","robos","secuestro","secuestros","clandestino","clandestina","clandestinas","clandestinos","García Harfuch","balean",
                  "delitos","Chapitos","Mayiza","arma","Marina","violencia","Comisión Nacional de Búsqueda", "personas desaparecidas","crimen organizado",
                 "Guardia Nacional","GN","narcotraficantes","disparó","dispararon","armas","aduana","ejecutan","narcorancho","asesinar","abaten","narcotraficante",
                 "decomiso","decomisan","detienen","detención", "desaparición forzada","Los Cantaritos","desapariciones","atentado","ejecutados",
                 "tiros","reclutamiento forzado", "exterminio","rifle","fusil","drogas","explosivos","detenidos","clandestino","narcotráfico","muertos",
                 "militares","exmilitares","marinos","tiroteo","madres buscadoras","desapariciones","cuerpos","raptadas","matan","asesinato","terrorismo",
                 "Semar","Seretaría de Marina","matanza","semefo","ejecutaron","ejecutan","balaceras","capos","delincuentes","abrazos no balazos", "mencho",
                 "derecho de piso","Los Alegres del Barranco","Doña Carlota","disparando","confiscan","rearma","incautan", "Teresa González Murillo",
                 "crimen", "familia michoacana","contrabando", "Carlota N", "sin vida","operativos", "balacera","roban","El mayo","restos humanos",
                 "mutilado", "linchan","linchamiento"],
    
    "Educación": ["Acceso a educación","SEP","Mario Delgado","CCH","UNAM","IPN","universitarios","escuelas", "maestros"],
    
    "Salud": ["salud", "IMSS", "IMSS Bienestar","tos ferina","sarampión","enfermedad",  "vacuna", "médico","medicina", "clínica", "epidemia","vacunación",
              "obesidad","vacunas","medicinas", "Eduardo Clark","Kershenobich","sobrepeso", "influenza", "H5N1", "aviar","dengue", "hospitales"]
}
palabras_positivas = ["éxito", "triunfó", "logro", "logros", "avance", "progreso", "mejoría", "crecimiento","baja desempleo","baja inflación",
    "decomiso de fentanilo","Suspenden Contingencia Ambiental","suspende contingencia ambiental","Suspendida la contingencia ambiental",
    "aprobación","suma otra inversión","felicidad","Guardia Nacional asegura","GN aseguran","felices","feliz","Golpe al narco",
    "decomiso","decomisan","firmeza de Sheinbaum","detienen","aseguran dosis","aseguran cargamento","aseguran artefactos","aprehenden","detenidos",
    "elementos navales descubren","presume Sheinbaum","Sheinbaum celebra","Sheinbaum inaugura","rescatan","Inaugura Sheinbaum","invierte",
    "incentivar","Entrega Clara Brugada","tendrá inversión","anuncia inversión","bienvenidas las inversiones","impulsa el desarrollo",
    "impulsa inversión","potencia minería","revoluciona la minería","Avanza la minería","minería del futuro","Más vivo",
    "impulsa la minería","amplía meta", "impulsan innovación", "transforma","innovación","auge","repunta","construcción sostenible", 
    "construcción sustentable","innovaciones","eficiencia","avanza","detonan construcción", "Obtiene reconocimiento", "fortalece",
    "señales de recuperación","crece el sector", "impulsa","bienvenidas","bienvenida","facilidades","vanguardia","inclusión",
    "inicia la construcción", "anuncian la construcción","anuncia proyectos","iniciará su construcción", "anuncia la construcción",
    "arranca construcción", "anuncia construcción","acelerará construcción", "acelera construcción", "acuerdan construcción",
    "inicia proyecto","agradece","innovadores","inicia construcción","nuevo hospital","nuevos hospitales","resalta construcción",
    "banderazo","anuncia una inversión", "iniciará construcción", "iniciará la construcción","inicio la construcción", 
    "inician construcción","optimismo","incautan","rescatan", "decomiso","decomisan","detención", "confiscan", "detenidos",
    "inversión millonaria", "beneficiará", "celebra", "golpe al peso", "Lanzan programas","forteleció el peso","fortalece le peso",
    "peso fortalecido"]

palabras_negativas = ["fracaso", "crisis", "colapso", "escándalo", "corrupción","irresponsabilidad", "problema","deterioro", "violencia", 
    "homicidio", "irrespetuosas", "peligroso", "peligrosa", "fallida","podrido", "difícil","dura", "duro", "incongruente", 
    "retrocede", "grave","traición", "pierde", "desaprobación", "desaprueba","engaña", "engaño", "sucio", "sucia", "perdería", 
    "fallido", "mal", "malo", "mala", "débil","violación","pesadilla", "desplazamiento", "polémica", "desapariciones", "cacería",
    "polémicas", "colapsó", "ineficiente", "agujero", "fallé", "desastre","vergüenza", "aumento del desempleo", "Mencho","Chapitos",
    "Mayiza","Teuchitlán","Izaguirre","exterminio","fosa clandestina","CJNG","Cártel de Sinaloa","Activan Contingencia Ambiental",
    "chantaje","chantajea","horror","mala calidad del aire","huachicol","fuga de gas","toman casetas","secuestrado","asesinados",
    "Izaguirre","contaminación","Ayotzinapa","balacera","infierno","Cartel Jalisco Nueva Generación","obesidad","sobrepeso","bombardeo",
    "sismo","tiroteo","incendio","muere","muerte","explosión","reconoce fallas","ejecutan","polvo","bloqueo","rapta","cuerpos","atentado",
    "sequía","asesinato","contra México","sanciones a México","aranceles","arancel", "terrorismo","terrorista","matanza","ejecutaron","ejecutan",
    "Descomposición","balaceras","Peso cae","narco mexicano incursiona","estafan","retroceso","cae","afectarán","afecta",
    "recesión","riesgo","desaceleración","retrasos","daños","Myanmar", "denuncian", "afectan", "ilelgal","devastación",
    "Protesta", "retrasaría","inconformidades","rechazan","clausura","desmienten construcción","suspender","frenar construcción",
    "protestan", "protestas","repudia","repudian","recortan expectativa", "Teuchitlán","Izaguirre","cárteles","cártel","homicidios",
    "Tren de Aragua","Cártel de Sinaloa","armado","violencia","baleado","fentanilo","narco","comando","asesinada","asesinado","asesinados","asesinadas",
    "asesinan","robo","robos","secuestro","secuestros","clandestino","clandestina","clandestinas","clandestinos","balean",
    "delitos","Chapitos","Mayiza","arma","Marina","violencia","Comisión Nacional de Búsqueda", "personas desaparecidas","crimen organizado",
    "Guardia Nacional","GN","narcotraficantes","disparó","dispararon","armas","aduana","ejecutan","narcorancho","asesinar","narcotraficante",
    "desaparición forzada","Los Cantaritos","desapariciones","atentado","ejecutados","homicidio","desaparecidos",
    "tiros","reclutamiento forzado", "exterminio","rifle","fusil","drogas","explosivos","clandestino","narcotráfico","muertos",
    "militares","exmilitares","marinos","tiroteo","madres buscadoras","desapariciones","raptadas","matan","asesinato","terrorismo",
    "matanza","semefo","ejecutaron","ejecutan","balaceras","capos","delincuentes","abrazos no balazos", "mencho",
    "derecho de piso","Los Alegres del Barranco","Doña Carlota","disparando","rearma","incautan", "Teresa González Murillo",
    "crimen", "familia michoacana","contrabando", "Carlota N", "sin vida","operativos", "balacera","roban","El mayo", "desaparecido","CJNG",
    "feminicidio", "aranceles recíprocos", "tira", "escasez de agua", "restos humanos", "mutilado", "linchan","linchamiento", "Cae confianza"]

def clasificar_sentimiento(titulo, adjetivos):
    titulo_norm = normalizar(titulo)
    palabras_pos = [word for word in palabras_positivas if normalizar(word) in titulo_norm]
    palabras_neg = [word for word in palabras_negativas if normalizar(word) in titulo_norm]

    if palabras_pos and not palabras_neg:
        return "Positiva"
    elif palabras_neg and not palabras_pos:
        return "Negativa"
    else:
        return "Neutral"

def palabras_clasificadoras(titulo):
    titulo_norm = normalizar(titulo)
    return ", ".join([word for word in palabras_positivas + palabras_negativas if normalizar(word) in titulo_norm])

def categorizar_noticia(titulo):
    titulo_norm = normalizar(titulo)
    categorias_encontradas = set()
    for categoria, palabras in categorias.items():
        for palabra in palabras:
            palabra_norm = normalizar(palabra)
            if len(palabra_norm) <= 3:
                if f" {palabra_norm} " in f" {titulo_norm} ":
                    categorias_encontradas.add(categoria)
            else:
                if palabra_norm in titulo_norm:
                    categorias_encontradas.add(categoria)
    return ", ".join(categorias_encontradas) if categorias_encontradas else "Otros"

# Cargar catálogo de visitas mensuales
cat_path = "catalogo_medios_mexicanos_con reach.xlsx"
cat_df = pd.read_excel(cat_path)
cat_df["Dominio Limpio"] = cat_df["Dominio"].apply(lambda x: urlparse("https://" + str(x)).netloc.replace("www.", "").split("/")[0])
cat_df = cat_df.rename(columns={"Visitas mensuales con base en marzo 2025": "Visitas_Mensuales"})

noticias = []
ahora = datetime.now(timezone.utc)

for termino in terminos_busqueda:
    print(f"\n🔍 Buscando: {termino}")

    for medio in medios:
        dominio = urlparse("https://" + medio).netloc
        url = f"https://news.google.com/rss/search?q={termino}+site:{dominio}&hl=es-419&gl=MX&ceid=MX:es"

        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"⚠️ Error al obtener noticias de {dominio}")
            continue

        soup = BeautifulSoup(response.content, "xml")
        items = soup.find_all("item")
        if debug:
            print(f"⚙️  {dominio} → {len(items)} noticias")

        for item in items:
            title = item.title.text.strip()
            link = item.link.text.strip()
            description = item.description.text.strip() if item.description else ""
            pub_date_str = item.pubDate.text.strip()

            try:
                pub_date = parser.parse(pub_date_str)
            except Exception:
                continue

            if modo == "exacto":
                if pub_date.date() != fecha_objetivo:
                    continue
            elif modo == "rango":
                if ahora - pub_date > timedelta(days=dias_rango):
                    continue

            contenido = f"{title} {description}".lower()
            if termino.lower() in contenido:
                if debug:
                    print(f"📰 {title} | {pub_date.date()} | {dominio}")
                noticias.append([pub_date_str, title, link, dominio, termino])

# 📁 Crear carpeta si no existe
carpeta_salida = "noticias"
os.makedirs(carpeta_salida, exist_ok=True)
# Guardar resultados
if noticias:
    df = pd.DataFrame(noticias, columns=["Fecha", "Título", "Enlace", "Fuente", "Término"])
    df["Título"] = df["Título"].str.split(" - ").str[0]
    df = df.drop_duplicates(subset=["Título", "Fuente"])
    df["Fecha"] = pd.to_datetime(df["Fecha"], errors="coerce")
    df = df[df["Fecha"].notna()]
    df["Fecha"] = df["Fecha"].dt.strftime("%d/%m/%y")
    df["Adjetivos"] = df["Título"].apply(extraer_adjetivos)
    df["Sentimiento"] = df.apply(lambda row: clasificar_sentimiento(row["Título"], row["Adjetivos"]), axis=1)
    df["Palabras Clave"] = df["Título"].apply(palabras_clasificadoras)
    df["Categoría"] = df["Título"].apply(categorizar_noticia)
    df["Dominio Limpio"] = df["Fuente"].apply(lambda x: urlparse("https://" + str(x)).netloc.replace("www.", "").split("/")[0])
    df = df.merge(cat_df[["Dominio Limpio", "Visitas_Mensuales"]], on="Dominio Limpio", how="left")
    df["Reach estimado"] = df["Visitas_Mensuales"] * 0.001 * np.random.uniform(0.6, 1.2, size=len(df))

    sufijo_fecha = fecha_objetivo.strftime("%d-%m-%Y") if modo == "exacto" else ahora.strftime("%d-%m-%Y")
    nombre_archivo = os.path.join(
        carpeta_salida, f"noticias_provincia_{sufijo_fecha}.csv"
)
    df.to_csv(nombre_archivo, index=False, encoding="utf-8-sig")
    print(f"\n✅ Archivo guardado como '{nombre_archivo}' con {len(df)} noticias")
else:
    print("\n❌ No se encontraron noticias que cumplan los filtros.")



🔍 Buscando: Parque Industrial
⚙️  abcnoticias.mx → 100 noticias
📰 Se registra incendio en una tarimera del Parque Industrial Escobedo - ABCNoticias.mx | 2025-11-04 | abcnoticias.mx
⚙️  abcnoticias.mx → 100 noticias
📰 Se registra incendio en una tarimera del Parque Industrial Escobedo - ABCNoticias.mx | 2025-11-04 | abcnoticias.mx
⚙️  elnorte.com → 100 noticias
⚙️  info7.mx → 100 noticias
⚙️  elhorizonte.mx → 100 noticias
⚙️  elporvenir.mx → 100 noticias
⚙️  elregio.com → 69 noticias
⚙️  omareli.com → 9 noticias
⚙️  elmanana.com → 100 noticias
⚙️  latarde.com.mx → 100 noticias
⚙️  laprensa.mx → 100 noticias
⚙️  valledelnorte.com.mx → 2 noticias
⚙️  psn.si → 9 noticias
⚙️  jornadabc.com.mx → 17 noticias
⚙️  afntijuana.info → 100 noticias
⚙️  puntonorte.info → 100 noticias
⚙️  zetatijuana.com → 100 noticias
⚙️  diario.mx → 100 noticias
⚙️  netnoticias.mx → 100 noticias
⚙️  puentelibre.mx → 100 noticias
⚙️  nortedigital.mx → 100 noticias
⚙️  adiario.mx → 32 noticias
⚙️  juarezhoy.com.mx →

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, timezone
from urllib.parse import urlparse
from dateutil import parser
import unicodedata
import spacy
import numpy as np
import os


# === CONFIGURACIÓN GENERAL ===
terminos_busqueda = ["tariff", "tariffs","trade deal","trade deals","merger", "mergers", "auto industry","auto parts","supply chain",
                    "car industry","interest rates","the fed","federal reserve"]
                    
modo = "rango"  # Opciones: "exacto" o "rango"
debug = True     # True para imprimir detalles por noticia
dias_rango = 1   # Solo si modo = "rango"
fecha_objetivo = datetime(2025, 7,3).date()  # Solo si modo = "exacto"

medios = ["nytimes.com","washingtonpost.com", "wsj.com/", "bloomberg.com/", "reuters.com/", "politico.com/", "thehill.com/", "axios.com/", "cnbc.com/", 
          "cnn.com/", "www.foxnews.com", "nbcnews.com/", "abcnews.go.com/", "cbsnews.com/", "latimes.com/", "usatoday.com/", "npr.org/", "apnews.com/", "forbes.com/",
          "foreignpolicy.com/", "foxbusiness.com", "foreignaffairs.com/", "defensenews.com/", "lawfaremedia.org/", "brookings.edu/blog/", "heritage.org/",
          "theatlantic.com","newyorker.com","www.aljazeera.com","cgtn.com/","japantimes.co.jp/","latimes.com/","miamiherald.com/","bostonglobe.com",
          "fortune.com/","marketwatch.com/","www.forbes.com"]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
}

nlp = spacy.load("es_core_news_sm")

# Funciones auxiliares
def normalizar(texto):
    if not isinstance(texto, str):
        return ""
    texto = unicodedata.normalize("NFKD", texto)
    texto = texto.encode("ascii", "ignore").decode("utf-8")
    return texto.lower()

def extraer_adjetivos(texto):
    doc = nlp(str(texto))
    return [token.text for token in doc if token.pos_ == "ADJ"]

categorias = {"Interior": ["Rosa Icela Rodríguez", "SEGOB", "segob", "Secretaría de Gobernación", "corridas de toros","taurino", "tauromaquia", "ISSSTE",
                 "CNTE","SNTE","bloqueos","marchas","bloqueo","marcha", "INAI","Rocha Moya", "Mexicable", "Bienestar", "Sindicato",
                "jefe de gabinete", "Martí Batres", "gabinete", "fiesta brava", "San Lázaro","Cámara de Diputados","Sismo en México" 
                 "congreso", "partido", "senador", "gobernador", "senadora", "gobernadora", "Senado", "diputado","diputados","senadores", 
                 "PROFECO", "Infonavit","Sembrando Vida","desafuero","derechos laborales","reforma laboral","líder sindical","Noroña","Alito",
                 "Ricardo Anaya","Morena","PRI","PAN","Movimiento Ciudadano", "Comisión de Igualdad de Género","jubilación","Marko Cortés",
                 "Jóvenes construyendo","ley","leyes","reforma","reformas", "Kuri", "Menchaca","Lemus","Samuel García","UdeG","UNAM",
                "Evelyn Salgado","parlamento abierto","Mara Lezama","jucopo","Murat","morenistas","yunes","Monreal","Adán Augusto",
                "Armenta","PAN","Ivonne Ortega", "cuauhtémoc", "cuau", "pedro haces","ramírez cuellar","marín","rectora","ealy","desalojo",
                "colosio","anaya","Lajous","pensión","Ricardo Gallardo", "Programa alimentario","agroindustria", "Ramírez Bedolla","paro nacional",
                          "ventanilla digital", "conservadores", "asuntos internos", "Brugada","cdmx", "cablebús","Línea 1", "simulacro"],
    
    "Corrupción": ["corrupción", "corrupto", "desfalco", "soborno", "coludido","Odebrecht","sobreprecios","sobreprecio","ASF","Auditoría Superior de la Federación",
                  "irregularidades","sobrecosto","contratista","opacidad","compranet","Raquel Buenrostro","Andrea Chávez"],
    
    "Judicial": ["fiscalía", "cortes", "corte", "exfiscal", "audiencia", "fiscal", "extradición","juez", "jueza", "fiscalía general", "juicio",
                 "ministerio público","FGR","Lenia Batres","poder judicial","SCJN","judicial","extradición","extraditó","Aguakan","Gertz","Magistrada",
                "Tribunal Superior de Justicia","TSJ","Inés Gómez Mont", "Caso Zimapán","UIF","Yasmín Esquivel","Lenia Batres","Loreta Ortíza"],
    
    "Electoral": ["elección", "campaña", "elecciones", "candidato", "candidatos", "candidata","TEPJF","OPLES","INE","aprehenden","candidaturas"],  
    
    "Medio Ambiente":["contingencia","incendio forestal","incendios forestales","deforestación","racionamiento","escasez de agua","sin agua",
                      "ozono","contaminación","crisis climática","desastres climáticos","huracán","huracanes","corte de agua","calidad del aire",
                     "vaquita marina","Áreas Naturales Protegidas","PROFEPA","polvo","aire sucio","sequía","Plan Nacional Hídrico","acceso al agua",
                      "calor extremo","agua","SEMARNAT","racionamiento","glaciares","greenpeace","totoabas","tortugas", "Frente Frío", "Medio Ambiente",
                     "temporada de calor","radiación solar"],
    
    "Exterior": ["canciller", "TLC", "Tratado de Libre Comercio", "diplomático", "diplomática", "embajada", "embajador", "TMEC", "Venezuela", "ONU", 
                 "Nicolás Maduro","OTAN", "Frontera", "deportaciones","deportados","deportado", "Zelenski","Ucrania", "Palestina", "Hamás","Hamas", "Israel", 
                 "Lula da Silva","Putin","Papa Francisco","Millei","Vaticano", "Chile","Hungría","UE","Unión Europea","Francia","Macron", "Canadá", "Guatemala",
                 "FMI","Kennedy","Donald Trump","Trump","Marco Rubio","De la Fuente","Musk","Wall Street","Fed","EU","Gaza","Estados Unidos","EEUU","Bukele",
                "speak","Estambul","Turquía","Arabia Saudita","Ted Cruz","Sahel","relación bilateral","Carney", "Kristi Noem","Lutnick","migración",
                "celac","Honduras","Petro","Xiomara","Kamala", "Colombia","Von Der Leyen","China","chino","Corea del Sur","Mangione","Myanmar","EE.UU.",
                "Noboa","Ecuador","Noem","antiinmigrantes","SRE","Norteamérica","extranjeros", "extranjero","Emiratos Árabes"],
    
    "Economía": ["Desempleo","nearshoring","SAT","Secretaría de Hacienda","Secretario de Hacienda","Presupuesto","fiscal","arancel", "aranceles"
                "exportación","importación","exportaciones","importaciones","precios","cotización","tipo de cambio","CEO","CFE","General Motors",
                 "energética","crudo","Banxico","Banco de México", "tasas de interés","tasa de interés","peso mexicano", "armadoras", "automotriz",
                 "competitividad","calificadora","calificadoras","S&P","Moody's","Fitch","Nasdaq", "Dow Jones","Plan México","trigo","PIB","oro",
                "aduanero","aduanera","déficit","Pemex","hidrocarburos","KPMG","Deloitte","Santander","sofipos","nóminas","recesión", "empleo","remesas",
                "derrama económica","Ebrard","fintech","Banco","Bancos","DOGE","Coppel","Walmart","Oxxo","reforma energética","turistas","telmex",
                "Kueski","SENER","AIFA","T-MEC","BYD","Bolsa mexicana","deprecia","CAINTRA","ronda de inversión","inversión millonaria","Política económica","Italika",
                "AIMMGM","consumo privado","Empleo","Hongqi","inversión conjunta","Coca Cola","Topo Chico","precio de los huevos","tendrá inversión","ADM","Peso cae",
                "fovissste","Slim","tortilla","tortillas","CEO","Altagracia","recaudación fiscal","economía", "Mercedes-Benz", "Volkswagen","económico",
                "VW","Audi","BMW","stellantis","gm","general motors","dos bocas","refinería","Transístmico","tarifas","acero","aluminio", "Nissan",
                "automotrices","agrícola", "farmacéutica","interoceánico", "Amazon","crisis económica","chrysler", "salinas pliego","volvo","citi",
                "cce","shcp","infonavit","comercial","Land Rover","Jaguar","Oracle","JPMorgan Chase","Truth Social","Totalplay","fabricación de autos",
                "cierra dólar","expectativa de crecimiento","BMV","Bolsa Mexicana de Valores", "peñoles", "hacienda","unicornio", "recaudación","INEGI",
                "confianza del consumidor","cosechando soberanía","ocupación hotelera","económica","vías férreas","BM","monopolio","antimonopolio",
                "monopolios","antimonopolios","cofece","leche para el bienestar", "miel del bienestar","cepal"],
    
    "Seguridad": ["Teuchitlán","Izaguirre","cárteles","cártel","homicidios","homicidio","desaparecidos","desaparecido","CJNG","feminicidio",
                  "Tren de Aragua","Cártel de Sinaloa","armado","violencia","baleado","fentanilo","narco","comando","asesinada","asesinado","asesinados","asesinadas",
                  "asesinan","robo","robos","secuestro","secuestros","clandestino","clandestina","clandestinas","clandestinos","García Harfuch","balean",
                  "delitos","Chapitos","Mayiza","arma","Marina","violencia","Comisión Nacional de Búsqueda", "personas desaparecidas","crimen organizado",
                 "Guardia Nacional","GN","narcotraficantes","disparó","dispararon","armas","aduana","ejecutan","narcorancho","asesinar","abaten","narcotraficante",
                 "decomiso","decomisan","detienen","detención", "desaparición forzada","Los Cantaritos","desapariciones","atentado","ejecutados",
                 "tiros","reclutamiento forzado", "exterminio","rifle","fusil","drogas","explosivos","detenidos","clandestino","narcotráfico","muertos",
                 "militares","exmilitares","marinos","tiroteo","madres buscadoras","desapariciones","cuerpos","raptadas","matan","asesinato","terrorismo",
                 "Semar","Seretaría de Marina","matanza","semefo","ejecutaron","ejecutan","balaceras","capos","delincuentes","abrazos no balazos", "mencho",
                 "derecho de piso","Los Alegres del Barranco","Doña Carlota","disparando","confiscan","rearma","incautan", "Teresa González Murillo",
                 "crimen", "familia michoacana","contrabando", "Carlota N", "sin vida","operativos", "balacera","roban","El mayo","restos humanos",
                 "mutilado", "linchan","linchamiento"],
    
    "Educación": ["Acceso a educación","SEP","Mario Delgado","CCH","UNAM","IPN","universitarios","escuelas", "maestros"],
    
    "Salud": ["salud", "IMSS", "IMSS Bienestar","tos ferina","sarampión","enfermedad",  "vacuna", "médico","medicina", "clínica", "epidemia","vacunación",
              "obesidad","vacunas","medicinas", "Eduardo Clark","Kershenobich","sobrepeso", "influenza", "H5N1", "aviar","dengue", "hospitales"]
}
palabras_positivas = ["éxito", "triunfó", "logro", "logros", "avance", "progreso", "mejoría", "crecimiento","baja desempleo","baja inflación",
    "decomiso de fentanilo","Suspenden Contingencia Ambiental","suspende contingencia ambiental","Suspendida la contingencia ambiental",
    "aprobación","suma otra inversión","felicidad","Guardia Nacional asegura","GN aseguran","felices","feliz","Golpe al narco",
    "decomiso","decomisan","firmeza de Sheinbaum","detienen","aseguran dosis","aseguran cargamento","aseguran artefactos","aprehenden","detenidos",
    "elementos navales descubren","presume Sheinbaum","Sheinbaum celebra","Sheinbaum inaugura","rescatan","Inaugura Sheinbaum","invierte",
    "incentivar","Entrega Clara Brugada","tendrá inversión","anuncia inversión","bienvenidas las inversiones","impulsa el desarrollo",
    "impulsa inversión","potencia minería","revoluciona la minería","Avanza la minería","minería del futuro","Más vivo",
    "impulsa la minería","amplía meta", "impulsan innovación", "transforma","innovación","auge","repunta","construcción sostenible", 
    "construcción sustentable","innovaciones","eficiencia","avanza","detonan construcción", "Obtiene reconocimiento", "fortalece",
    "señales de recuperación","crece el sector", "impulsa","bienvenidas","bienvenida","facilidades","vanguardia","inclusión",
    "inicia la construcción", "anuncian la construcción","anuncia proyectos","iniciará su construcción", "anuncia la construcción",
    "arranca construcción", "anuncia construcción","acelerará construcción", "acelera construcción", "acuerdan construcción",
    "inicia proyecto","agradece","innovadores","inicia construcción","nuevo hospital","nuevos hospitales","resalta construcción",
    "banderazo","anuncia una inversión", "iniciará construcción", "iniciará la construcción","inicio la construcción", 
    "inician construcción","optimismo","incautan","rescatan", "decomiso","decomisan","detención", "confiscan", "detenidos",
    "inversión millonaria", "beneficiará", "celebra", "golpe al peso", "Lanzan programas","forteleció el peso","fortalece le peso",
    "peso fortalecido"]

palabras_negativas = ["fracaso", "crisis", "colapso", "escándalo", "corrupción","irresponsabilidad", "problema","deterioro", "violencia", 
    "homicidio", "irrespetuosas", "peligroso", "peligrosa", "fallida","podrido", "difícil","dura", "duro", "incongruente", 
    "retrocede", "grave","traición", "pierde", "desaprobación", "desaprueba","engaña", "engaño", "sucio", "sucia", "perdería", 
    "fallido", "mal", "malo", "mala", "débil","violación","pesadilla", "desplazamiento", "polémica", "desapariciones", "cacería",
    "polémicas", "colapsó", "ineficiente", "agujero", "fallé", "desastre","vergüenza", "aumento del desempleo", "Mencho","Chapitos",
    "Mayiza","Teuchitlán","Izaguirre","exterminio","fosa clandestina","CJNG","Cártel de Sinaloa","Activan Contingencia Ambiental",
    "chantaje","chantajea","horror","mala calidad del aire","huachicol","fuga de gas","toman casetas","secuestrado","asesinados",
    "Izaguirre","contaminación","Ayotzinapa","balacera","infierno","Cartel Jalisco Nueva Generación","obesidad","sobrepeso","bombardeo",
    "sismo","tiroteo","incendio","muere","muerte","explosión","reconoce fallas","ejecutan","polvo","bloqueo","rapta","cuerpos","atentado",
    "sequía","asesinato","contra México","sanciones a México","aranceles","arancel", "terrorismo","terrorista","matanza","ejecutaron","ejecutan",
    "Descomposición","balaceras","Peso cae","narco mexicano incursiona","estafan","retroceso","cae","afectarán","afecta",
    "recesión","riesgo","desaceleración","retrasos","daños","Myanmar", "denuncian", "afectan", "ilelgal","devastación",
    "Protesta", "retrasaría","inconformidades","rechazan","clausura","desmienten construcción","suspender","frenar construcción",
    "protestan", "protestas","repudia","repudian","recortan expectativa", "Teuchitlán","Izaguirre","cárteles","cártel","homicidios",
    "Tren de Aragua","Cártel de Sinaloa","armado","violencia","baleado","fentanilo","narco","comando","asesinada","asesinado","asesinados","asesinadas",
    "asesinan","robo","robos","secuestro","secuestros","clandestino","clandestina","clandestinas","clandestinos","balean",
    "delitos","Chapitos","Mayiza","arma","Marina","violencia","Comisión Nacional de Búsqueda", "personas desaparecidas","crimen organizado",
    "Guardia Nacional","GN","narcotraficantes","disparó","dispararon","armas","aduana","ejecutan","narcorancho","asesinar","narcotraficante",
    "desaparición forzada","Los Cantaritos","desapariciones","atentado","ejecutados","homicidio","desaparecidos",
    "tiros","reclutamiento forzado", "exterminio","rifle","fusil","drogas","explosivos","clandestino","narcotráfico","muertos",
    "militares","exmilitares","marinos","tiroteo","madres buscadoras","desapariciones","raptadas","matan","asesinato","terrorismo",
    "matanza","semefo","ejecutaron","ejecutan","balaceras","capos","delincuentes","abrazos no balazos", "mencho",
    "derecho de piso","Los Alegres del Barranco","Doña Carlota","disparando","rearma","incautan", "Teresa González Murillo",
    "crimen", "familia michoacana","contrabando", "Carlota N", "sin vida","operativos", "balacera","roban","El mayo", "desaparecido","CJNG",
    "feminicidio", "aranceles recíprocos", "tira", "escasez de agua", "restos humanos", "mutilado", "linchan","linchamiento", "Cae confianza"]

def clasificar_sentimiento(titulo, adjetivos):
    titulo_norm = normalizar(titulo)
    palabras_pos = [word for word in palabras_positivas if normalizar(word) in titulo_norm]
    palabras_neg = [word for word in palabras_negativas if normalizar(word) in titulo_norm]

    if palabras_pos and not palabras_neg:
        return "Positiva"
    elif palabras_neg and not palabras_pos:
        return "Negativa"
    else:
        return "Neutral"

def palabras_clasificadoras(titulo):
    titulo_norm = normalizar(titulo)
    return ", ".join([word for word in palabras_positivas + palabras_negativas if normalizar(word) in titulo_norm])

def categorizar_noticia(titulo):
    titulo_norm = normalizar(titulo)
    categorias_encontradas = set()
    for categoria, palabras in categorias.items():
        for palabra in palabras:
            palabra_norm = normalizar(palabra)
            if len(palabra_norm) <= 3:
                if f" {palabra_norm} " in f" {titulo_norm} ":
                    categorias_encontradas.add(categoria)
            else:
                if palabra_norm in titulo_norm:
                    categorias_encontradas.add(categoria)
    return ", ".join(categorias_encontradas) if categorias_encontradas else "Otros"

# Cargar catálogo de visitas mensuales
cat_path = "catalogo_medios_mexicanos_con reach.xlsx"
cat_df = pd.read_excel(cat_path)
cat_df["Dominio Limpio"] = cat_df["Dominio"].apply(lambda x: urlparse("https://" + str(x)).netloc.replace("www.", "").split("/")[0])
cat_df = cat_df.rename(columns={"Visitas mensuales con base en marzo 2025": "Visitas_Mensuales"})

noticias = []
ahora = datetime.now(timezone.utc)

for termino in terminos_busqueda:
    print(f"\n🔍 Buscando: {termino}")

    for medio in medios:
        dominio = urlparse("https://" + medio).netloc
        url = f"https://news.google.com/rss/search?q={termino}+site:{dominio}&hl=en&gl=US&ceid=US:en"

        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"⚠️ Error al obtener noticias de {dominio}")
            continue

        soup = BeautifulSoup(response.content, "xml")
        items = soup.find_all("item")
        if debug:
            print(f"⚙️  {dominio} → {len(items)} noticias")

        for item in items:
            title = item.title.text.strip()
            link = item.link.text.strip()
            description = item.description.text.strip() if item.description else ""
            pub_date_str = item.pubDate.text.strip()

            try:
                pub_date = parser.parse(pub_date_str)
            except Exception:
                continue

            if modo == "exacto":
                if pub_date.date() != fecha_objetivo:
                    continue
            elif modo == "rango":
                if ahora - pub_date > timedelta(days=dias_rango):
                    continue

            contenido = f"{title} {description}".lower()
            if termino.lower() in contenido:
                if debug:
                    print(f"📰 {title} | {pub_date.date()} | {dominio}")
                noticias.append([pub_date_str, title, link, dominio, termino])

# 📁 Crear carpeta si no existe
carpeta_salida = "noticias"
os.makedirs(carpeta_salida, exist_ok=True)
# Guardar resultados
if noticias:
    df = pd.DataFrame(noticias, columns=["Fecha", "Título", "Enlace", "Fuente", "Término"])
    df["Título"] = df["Título"].str.split(" - ").str[0]
    df = df.drop_duplicates(subset=["Título", "Fuente"])
    df["Fecha"] = pd.to_datetime(df["Fecha"], errors="coerce")
    df = df[df["Fecha"].notna()]
    df["Fecha"] = df["Fecha"].dt.strftime("%d/%m/%y")
    df["Adjetivos"] = df["Título"].apply(extraer_adjetivos)
    df["Sentimiento"] = df.apply(lambda row: clasificar_sentimiento(row["Título"], row["Adjetivos"]), axis=1)
    df["Palabras Clave"] = df["Título"].apply(palabras_clasificadoras)
    df["Categoría"] = df["Título"].apply(categorizar_noticia)
    df["Dominio Limpio"] = df["Fuente"].apply(lambda x: urlparse("https://" + str(x)).netloc.replace("www.", "").split("/")[0])
    df = df.merge(cat_df[["Dominio Limpio", "Visitas_Mensuales"]], on="Dominio Limpio", how="left")
    df["Reach estimado"] = df["Visitas_Mensuales"] * 0.001 * np.random.uniform(0.6, 1.2, size=len(df))

    sufijo_fecha = fecha_objetivo.strftime("%d-%m-%Y") if modo == "exacto" else ahora.strftime("%d-%m-%Y")
    nombre_archivo = os.path.join(
        carpeta_salida, f"noticias_us_{sufijo_fecha}.csv"
)
    df.to_csv(nombre_archivo, index=False, encoding="utf-8-sig")
    print(f"\n✅ Archivo guardado como '{nombre_archivo}' con {len(df)} noticias")
else:
    print("\n❌ No se encontraron noticias que cumplan los filtros.")



🔍 Buscando: tariff
⚙️  nytimes.com → 100 noticias
📰 Bessent Plans to Attend Tariff Arguments at the Supreme Court - The New York Times | 2025-11-04 | nytimes.com
📰 She Voted for Trump Three Times. Now She Is Leading a Fight Against His Tariffs. - The New York Times | 2025-11-04 | nytimes.com
📰 Opinion | Why the Trump Tariffs Supreme Court Case Could Be Fun Listening - The New York Times | 2025-11-04 | nytimes.com
📰 China’s Global Exports Continue to Grow Despite Trump Tariffs - The New York Times | 2025-11-04 | nytimes.com
📰 Trump’s Tariffs and Push Against Limits Face Election and Court Tests - The New York Times | 2025-11-04 | nytimes.com
📰 Opinion | Trump’s China Trade and Tariff Policy Is a Hot Mess - The New York Times | 2025-11-03 | nytimes.com
📰 Will Trump’s Tariff Deal Tilt the Playing Field Back Toward China? - The New York Times | 2025-11-03 | nytimes.com
⚙️  washingtonpost.com → 100 noticias
📰 Trump’s foe in tariffs case? A legal group funded by conservatives. - The Washing